In [ ]:
!pip install ipython-sql

# The code install the ipython-sql package using the pip package manager.
# pip is a tool used to install and manage Python packages. 
# The ! symbol at the beginning of the line is used in Jupyter notebooks to run shell commands.

In [1]:
import pandas as pd
import sqlite3

In [2]:
%load_ext sql

# This is a Jupyter notebook magic command that loads an SQL extension.
# This allows user to execute SQL queries directly in the Jupyter notebook.
# We can connect to any database which is supported by SQLAlchemy, in this project we will use SQLite database.

In [3]:
%sql sqlite:///MovieNow.db
# %sql is the magic command to connect to a SQLite database.
# The command sets the connection string to the default SQLite database.

##### Creating a database

In [ ]:
# Create blank tables inside SQLite database
%sql --file ./MovieNow-sqlite.sql

In [4]:
# Load Actors CSV file into a DataFrame
actors = 'actors.csv'
actsin = 'actsin.csv'
customers = 'customers.csv'
movies = 'movies.csv'
renting = 'renting.csv'

df_at = pd.read_csv(actors)
df_as = pd.read_csv(actsin)
df_c = pd.read_csv(customers)
df_m = pd.read_csv(movies)
df_r = pd.read_csv(renting)

# Connect to SQLite database
conn = sqlite3.connect('MovieNow.db')

# Write DataFrame to SQLite table
df_at.to_sql('actors', conn, index=False, if_exists='replace')
df_as.to_sql('actsin', conn, index=False, if_exists='replace')
df_c.to_sql('customers', conn, index=False, if_exists='replace')
df_m.to_sql('movies', conn, index=False, if_exists='replace')
df_r.to_sql('renting', conn, index=False, if_exists='replace')

# Close the connection
conn.close()

##### Showing the tables

In [5]:
%%sql
SELECT *
FROM actors
LIMIT 5;

 * sqlite:///MovieNow.db
Done.


actor_id,name,year_of_birth,nationality,gender
1,Abbie Cornish,1982.0,Australia,female
2,Adam Sandler,1966.0,USA,male
3,Al Pacino,1940.0,USA,male
4,Amy Adams,1974.0,USA,female
5,Andrea Riseborough,1981.0,British,female


In [7]:
%%sql
SELECT *
FROM actsin
LIMIT 5;

 * sqlite:///MovieNow.db
Done.


actsin_id,movie_id,actor_id
1,37,1
2,56,2
3,10,3
4,14,3
5,29,3


In [10]:
%%sql
SELECT *
FROM customers
LIMIT 5;

 * sqlite:///MovieNow.db
Done.


id,name,nationality,gender,dob,membersince
1,Robert Bohm,Austria,male,1980-07-30,2018-09-02
2,Wolfgang Ackermann,Austria,male,1971-11-17,2018-10-15
3,Daniela Herzog,Austria,female,1974-08-07,2019-02-14
4,Julia Jung,Austria,female,1991-01-04,2017-11-22
5,Juliane Kirsch,Austria,female,1977-03-01,2018-12-16


In [11]:
%%sql
SELECT *
FROM movies
LIMIT 5;

 * sqlite:///MovieNow.db
Done.


movie_id,title,genre,runtime,year_of_releas,renting_price
1,One Night at McCool's,Comedy,93,2001,2.09
2,Swordfish,Drama,99,2001,2.19
3,What Women Want,Comedy,127,2001,2.59
4,Training Day,Drama,122,2001,1.79
5,The Fellowship of the Ring,Science Fiction & Fantasy,178,2001,2.59


In [12]:
%%sql
SELECT *
FROM renting
LIMIT 5;

 * sqlite:///MovieNow.db
Done.


renting_id,customer_id,movie_id,rating,date_renting
1,41,8,None,2018-10-09
2,10,29,10.0,2017-03-01
3,108,45,4.0,2018-06-08
4,39,66,8.0,2018-10-22
5,104,15,7.0,2019-03-18


### Basic Information

#### What its the total number of the members?

In [20]:
%%sql
SELECT COUNT(DISTINCT name) AS number_of_members
FROM customers;

 * sqlite:///MovieNow.db
Done.


number_of_members
123


#### What is the total number of the movies?

In [38]:
%%sql
SELECT COUNT(DISTINCT title) AS number_of_movies
FROM movies;

 * sqlite:///MovieNow.db
Done.


number_of_movies
71


#### What is the income per year?

In [15]:
%%sql
SELECT strftime('%Y', date_renting) AS year,
       ROUND(SUM(renting_price),2) AS yearly_income
FROM renting AS r
LEFT JOIN movies AS m
ON r.movie_id = m.movie_id
GROUP BY strftime('%Y', date_renting)
ORDER BY year DESC;

 * sqlite:///MovieNow.db
Done.


year,yearly_income
2019,354.51
2018,658.02
2017,263.19


In [16]:
%%sql
SELECT MAX(date_renting)
FROM renting;

 * sqlite:///MovieNow.db
Done.


MAX(date_renting)
2019-05-01


In [21]:
%%sql
WITH quarter_categories AS (
    SELECT
        CASE
            WHEN strftime('%m', date_renting) BETWEEN '01' AND '03' THEN 'Q1'
            WHEN strftime('%m', date_renting) BETWEEN '04' AND '06' THEN 'Q2'
            WHEN strftime('%m', date_renting) BETWEEN '07' AND '09' THEN 'Q3'
            WHEN strftime('%m', date_renting) BETWEEN '10' AND '12' THEN 'Q4'
            ELSE 'Unknown'
        END AS quarter,
        renting_id
    FROM renting
)

SELECT 
    strftime('%Y', date_renting) AS year, 
    quarter,
    ROUND(SUM(renting_price),2) AS quarterly_income
FROM renting AS r
LEFT JOIN quarter_categories AS qc
ON r.renting_id = qc.renting_id
LEFT JOIN movies AS m
ON r.movie_id = m.movie_id
GROUP BY year, quarter
ORDER BY year DESC, quarter DESC;

 * sqlite:///MovieNow.db
Done.


year,quarter,quarterly_income
2019,Q2,91.18
2019,Q1,263.33
2018,Q4,203.47
2018,Q3,180.39
2018,Q2,186.95
2018,Q1,87.21
2017,Q4,100.95
2017,Q3,90.67
2017,Q2,50.67
2017,Q1,20.9


Movie rent per year and quarter:

In [16]:
%%sql
SELECT strftime('%Y', date_renting) AS year,
       COUNT(renting_id) AS n_renting
FROM renting
GROUP BY strftime('%Y', date_renting);

 * sqlite:///MovieNow.db
Done.


year,n_renting
2017,121
2018,298
2019,159


In [23]:
%%sql
WITH quarter_categories AS (
    SELECT
        CASE
            WHEN strftime('%m', date_renting) BETWEEN '01' AND '03' THEN 'Q1'
            WHEN strftime('%m', date_renting) BETWEEN '04' AND '06' THEN 'Q2'
            WHEN strftime('%m', date_renting) BETWEEN '07' AND '09' THEN 'Q3'
            WHEN strftime('%m', date_renting) BETWEEN '10' AND '12' THEN 'Q4'
            ELSE 'Unknown'
        END AS quarter,
        strftime('%Y', date_renting) AS year,
        renting_id
    FROM renting
)

SELECT year, quarter, COUNT(renting_id) AS n_renting
FROM quarter_categories
GROUP BY year, quarter
ORDER BY year DESC, quarter DESC;

 * sqlite:///MovieNow.db
Done.


year,quarter,n_renting
2019,Q2,42
2019,Q1,117
2018,Q4,93
2018,Q3,81
2018,Q2,85
2018,Q1,39
2017,Q4,45
2017,Q3,43
2017,Q2,23
2017,Q1,10


Highly rented movie in 2018:

In [5]:
%%sql
SELECT title, COUNT(renting_id) AS n_renting
FROM renting AS r
LEFT JOIN movies AS m
ON r.movie_id = m.movie_id
WHERE date_renting BETWEEN '201-01-01' AND '2019-12-31'
GROUP BY r.movie_id
ORDER BY n_renting DESC
LIMIT 10;

 * sqlite:///MovieNow.db
Done.


title,n_renting
World Trade Center,7
Young Adult,5
Fair Game,5
Harry Potter and the Half-Blood Prince,5
No Country for Old Men,5
Django Unchained,4
The Fighter,4
Morning Glory,4
The Kingdom,4
Capote,4


### Movie details

#### What are the top 5 rented movies?

In [5]:
%%sql
SELECT title, COUNT(renting_id) AS n_rented, COUNT(rating) AS n_rating
FROM renting
LEFT JOIN movies
ON renting.movie_id = movies.movie_id
GROUP BY renting.movie_id
ORDER BY n_rented DESC
LIMIT 5;

 * sqlite:///MovieNow.db
Done.


title,n_rented,n_rating
The Kingdom,15,10
Training Day,14,9
Harry Potter and the Half-Blood Prince,13,11
World Trade Center,13,8
Monster,13,7


In [16]:
%%sql
SELECT title, COUNT(rating) AS n_rated
FROM renting
LEFT JOIN movies
ON renting.movie_id = movies.movie_id
GROUP BY renting.movie_id
ORDER BY n_rated DESC
LIMIT 5;

 * sqlite:///MovieNow.db
Done.


title,n_rated
Harry Potter and the Half-Blood Prince,11
The Kingdom,10
Training Day,9
Winter's Bone,8
World Trade Center,8


#### What is the top 5 movies that have the highest average rating?
Note: only choose movie that has more than the average number of rating

In [33]:
%%sql
WITH count_rating AS (
    SELECT COUNT(rating) AS n_rating
    FROM renting
    GROUP BY movie_id
)
SELECT AVG(n_rating)
FROM count_rating

 * sqlite:///MovieNow.db
Done.


AVG(n_rating)
4.619718309859155


In [7]:
%%sql
WITH count_rating AS (
    SELECT COUNT(rating) AS n_rating
    FROM renting
    GROUP BY movie_id
)

SELECT title,
       COUNT(rating) AS number_rating,
       ROUND(AVG(rating),2) AS avg_rating
FROM movies AS m
LEFT JOIN renting AS r
ON m.movie_id = r.movie_id
GROUP BY r.movie_id
HAVING number_rating > (
    SELECT ROUND(AVG(n_rating),2)
    FROM count_rating
)
ORDER BY avg_rating DESC, number_rating DESC
LIMIT 5;

 * sqlite:///MovieNow.db
Done.


title,number_rating,avg_rating
No Country for Old Men,5,9.6
Django Unchained,6,9.33
Secondhand Lions,5,9.0
Ghost Rider: Spirit of Vengeance,5,9.0
Young Adult,6,8.83


#### What are movies that have rating above average?

In [30]:
%%sql
SELECT m.title, AVG(r.rating) AS avg_movie_rating, COUNT(rating) AS n_rating
FROM renting AS r
JOIN movies AS m ON r.movie_id = m.movie_id
GROUP BY m.title
ORDER BY avg_movie_rating DESC;


 * sqlite:///MovieNow.db
Done.


title,avg_movie_rating,n_rating
Astro Boy,10.0,2
The Fellowship of the Ring,9.75,4
No Country for Old Men,9.6,5
Django Unchained,9.333333333333334,6
What Women Want,9.0,2
Secondhand Lions,9.0,5
Ghost Rider: Spirit of Vengeance,9.0,5
Young Adult,8.833333333333334,6
Harry Potter and the Philosopher's Stone,8.8,5
Fool's Gold,8.75,4


#### How much income did each movie generate? 

In [70]:
%%sql
SELECT rm.title,
       ROUND(SUM(rm.renting_price),2) AS income_movie
FROM
       (SELECT m.title,  
               m.renting_price
       FROM renting AS r
       LEFT JOIN movies AS m
       ON r.movie_id=m.movie_id) AS rm
GROUP BY rm.title
ORDER BY income_movie DESC
LIMIT 5; 

 * sqlite:///MovieNow.db
Done.


title,income_movie
Bridget Jones - The Edge of Reason,37.57
Fair Game,34.68
The Kingdom,31.35
Two for the Money,30.69
Simone,29.59


#### What is the popular genre among customers?

In [9]:
%%sql
SELECT genre, COUNT(renting_id) AS n_rented
FROM renting AS r
LEFT JOIN movies AS m
ON r.movie_id = m.movie_id
GROUP BY genre
ORDER BY n_rented DESC;

 * sqlite:///MovieNow.db
Done.


genre,n_rented
Drama,319
Science Fiction & Fantasy,95
Comedy,69
Mystery & Suspense,39
Other,20
Action & Adventure,16
Animation,12
Art House & International,8


In [12]:
%%sql
SELECT genre, COUNT(movie_id) AS n_movie
FROM movies
GROUP BY genre
ORDER BY n_movie DESC;

 * sqlite:///MovieNow.db
Done.


genre,n_movie
Drama,36
Science Fiction & Fantasy,11
Comedy,10
Mystery & Suspense,6
Action & Adventure,3
Other,2
Animation,2
Art House & International,1


### Customer details

In [34]:
%%sql
SELECT country, gender, COUNT(*)
FROM customers
GROUP BY CUBE (country, gender);

 * sqlite:///MovieNow.db
(sqlite3.OperationalError) no such function: CUBE
[SQL: SELECT country, gender, COUNT(*)
FROM customers
GROUP BY CUBE (country, gender);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


Grouping based on age:<br>
1970 - 1979<br>
1980 - 1989<br>
1990 - 1999<br>

In [18]:
%%sql
WITH date_ranges AS (
    SELECT
        CASE
            WHEN date_of_birth BETWEEN '1970-01-01' AND '1979-12-31' THEN '1970s'
            WHEN date_of_birth BETWEEN '1980-01-01' AND '1989-12-31' THEN '1980s'
            WHEN date_of_birth BETWEEN '1990-01-01' AND '1999-12-31' THEN '1990s'
            ELSE 'Other'
        END AS birth_decade,
        customer_id
    FROM customers
)

SELECT
    birth_decade,
    COUNT(*) AS people_count
FROM date_ranges
GROUP BY birth_decade;

 * sqlite:///MovieNow.db
Done.


birth_decade,people_count
1970s,38
1980s,34
1990s,51


Grouping based on country:

In [19]:
%%sql
SELECT country, COUNT(*) AS n_customer
FROM customers
GROUP BY country
ORDER BY n_customer DESC;

 * sqlite:///MovieNow.db
Done.


country,n_customer
Italy,27
Spain,16
Poland,13
France,13
Slovenia,11
Hungary,11
Great Britan,10
Denmark,7
Belgium,6
Austria,5


### Actors

#### Actors with most screenplays

In [29]:
%%sql
SELECT name, COUNT(*) n_screenplay, actor_id
FROM actors
RIGHT JOIN actsin
USING(actor_id)
GROUP by name
ORDER BY n_screenplay DESC
LIMIT 5;

 * sqlite:///MovieNow.db
Done.


name,n_screenplay,actor_id
Rupert Grint,8,123
Emma Watson,8,42
Daniel Radcliffe,8,32
Nicolas Cage,4,105
Jamie Foxx,4,64


Movies played by top actors:

In [41]:
%%sql
SELECT name, actor_id
FROM actors
WHERE name IN ('Rupert Grint', 'Emma Watson', 'Daniel Radcliffe');

 * sqlite:///MovieNow.db
Done.


name,actor_id
Daniel Radcliffe,32
Emma Watson,42
Nicolas Cage,105


In [56]:
%%sql
WITH screenplays AS (
    SELECT asn.actor_id, name, asn.movie_id, title
    FROM actsin AS asn
    LEFT JOIN movies AS m
    ON asn.movie_id = m.movie_id
    LEFT JOIN actors AS a
    ON asn.actor_id = a.actor_id
)

SELECT name AS actor_name, title AS movie_title
FROM screenplays
WHERE name IN ('Rupert Grint', 'Emma Watson', 'Daniel Radcliffe', 'Nicolas Cage', 'Jamie Foxx');

 * sqlite:///MovieNow.db
Done.


actor_name,movie_title
Daniel Radcliffe,Harry Potter and the Philosopher's Stone
Daniel Radcliffe,Harry Potter and the Chamber of Secrets
Daniel Radcliffe,Harry Potter and the Prisoner of Azkaban
Daniel Radcliffe,Harry Potter and the Goblet of Fire
Daniel Radcliffe,Harry Potter and the Order of the Phoenix
Daniel Radcliffe,Harry Potter and the Half-Blood Prince
Daniel Radcliffe,Harry Potter and the Deathly Hallows – Part 1
Daniel Radcliffe,Harry Potter and the Deathly Hallows – Part 2
Emma Watson,Harry Potter and the Philosopher's Stone
Emma Watson,Harry Potter and the Chamber of Secrets


In [17]:
%%sql
WITH movie_rank AS (
    SELECT
        title, m.movie_id,
        COUNT(renting_id) AS n_rented,
        RANK() OVER(ORDER BY COUNT(renting_id) DESC) AS rank
    FROM movies AS m
    RIGHT JOIN renting AS r
    ON m.movie_id = r.movie_id
    GROUP BY r.movie_id
)

SELECT title, rank
FROM movie_rank
WHERE movie_id IN (
    SELECT movie_id
    FROM actsin AS asn
    LEFT JOIN movies AS m
    USING(movie_id)
    LEFT JOIN actors AS a
    USING(actor_id)
    WHERE name IN ('Rupert Grint', 'Emma Watson', 'Daniel Radcliffe', 'Nicolas Cage', 'Jamie Foxx')
)

 * sqlite:///MovieNow.db
Done.


title,rank
The Kingdom,1
Harry Potter and the Half-Blood Prince,3
World Trade Center,3
Harry Potter and the Deathly Hallows – Part 2,7
Django Unchained,9
Harry Potter and the Prisoner of Azkaban,17
Ray,17
Harry Potter and the Deathly Hallows – Part 1,24
Harry Potter and the Order of the Phoenix,24
The Frozen Ground,33
